<a href="https://colab.research.google.com/github/minukjeong/python/blob/main/ObjectDetection_Faster_R_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 명령어와 텐서플로우 허브 설치

In [1]:
# 시스템 명령어 처리
import os
import pathlib

# 이미지 처리
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as img

# 파일 처리
import io
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from six.moves.urllib.request import urlopen

# 텐서플로우 허브
import tensorflow as tf
import tensorflow_hub as hub

tf.get_logger().setLevel('ERROR')

# 사람 키포인트 튜플목록 불러오기 
Faster R-CNN ResNet50 v1 640x640 사용

In [2]:
# @title Run this!!

def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: the file path to the image

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  image = None
  if(path.startswith('http')):
    response = urlopen(path)
    image_data = response.read()
    image_data = BytesIO(image_data)
    image = Image.open(image_data)
  else:
    image_data = tf.io.gfile.GFile(path, 'rb').read()
    image = Image.open(BytesIO(image_data))

  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (1, im_height, im_width, 3)).astype(np.uint8)


ALL_MODELS = {'Faster R-CNN ResNet50 V1 640x640' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet50_v1_640x640/1'}

COCO17_HUMAN_POSE_KEYPOINTS = [(0, 1),
 (0, 2),
 (1, 3),
 (2, 4),
 (0, 5),
 (0, 6),
 (5, 7),
 (7, 9),
 (6, 8),
 (8, 10),
 (5, 6),
 (5, 11),
 (6, 12),
 (11, 12),
 (11, 13),
 (13, 15),
 (12, 14),
 (14, 16)]

#시각화 도구
적절하게 감지된 상자  키포인트 및 세분화로 이미지를 시각화하기 위해 TensorFlow Object Detection API를 사용합니다. 설치를 위해 리포지토리를 복제합니다.

In [3]:
!git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 3328, done.
remote: Counting objects: 100% (3328/3328), done.
remote: Compressing objects: 100% (2753/2753), done.
remote: Total 3328 (delta 882), reused 1387 (delta 525), pack-reused 0
Receiving objects: 100% (3328/3328), 34.31 MiB | 27.02 MiB/s, done.
Resolving deltas: 100% (882/882), done.


#객체 인식 API 설치

In [4]:
# 리눅스 명령어
%%bash
sudo apt install -y protobuf-compiler
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install -q .

Reading package lists...
Building dependency tree...
Reading state information...
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.




  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
multiprocess 0.70.12.2 requires dill>=0.3.4, but you have dill 0.3.1.1 which is incompatible.
gym 0.17.3 requires cloudpickle<1.7.0,>=1.2.0, but you have cloudpickle 2.0.0 which is incompatible.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [5]:
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import ops as utils_ops

%matplotlib inline

#플로팅을 위해 레이블 맵 데이터 로드

In [6]:
PATH_TO_LABELS = './models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

#감지 모델 빌드, 사정 훈련된 모델 가중치 로드

In [7]:
#@title Model Selection { display-mode: "form", run: "auto" }
model_display_name = 'Faster R-CNN ResNet50 V1 640x640' # @param ['CenterNet HourGlass104 512x512','CenterNet HourGlass104 Keypoints 512x512','CenterNet HourGlass104 1024x1024','CenterNet HourGlass104 Keypoints 1024x1024','CenterNet Resnet50 V1 FPN 512x512','CenterNet Resnet50 V1 FPN Keypoints 512x512','CenterNet Resnet101 V1 FPN 512x512','CenterNet Resnet50 V2 512x512','CenterNet Resnet50 V2 Keypoints 512x512','EfficientDet D0 512x512','EfficientDet D1 640x640','EfficientDet D2 768x768','EfficientDet D3 896x896','EfficientDet D4 1024x1024','EfficientDet D5 1280x1280','EfficientDet D6 1280x1280','EfficientDet D7 1536x1536','SSD MobileNet v2 320x320','SSD MobileNet V1 FPN 640x640','SSD MobileNet V2 FPNLite 320x320','SSD MobileNet V2 FPNLite 640x640','SSD ResNet50 V1 FPN 640x640 (RetinaNet50)','SSD ResNet50 V1 FPN 1024x1024 (RetinaNet50)','SSD ResNet101 V1 FPN 640x640 (RetinaNet101)','SSD ResNet101 V1 FPN 1024x1024 (RetinaNet101)','SSD ResNet152 V1 FPN 640x640 (RetinaNet152)','SSD ResNet152 V1 FPN 1024x1024 (RetinaNet152)','Faster R-CNN ResNet50 V1 640x640','Faster R-CNN ResNet50 V1 1024x1024','Faster R-CNN ResNet50 V1 800x1333','Faster R-CNN ResNet101 V1 640x640','Faster R-CNN ResNet101 V1 1024x1024','Faster R-CNN ResNet101 V1 800x1333','Faster R-CNN ResNet152 V1 640x640','Faster R-CNN ResNet152 V1 1024x1024','Faster R-CNN ResNet152 V1 800x1333','Faster R-CNN Inception ResNet V2 640x640','Faster R-CNN Inception ResNet V2 1024x1024','Mask R-CNN Inception ResNet V2 1024x1024']
model_handle = ALL_MODELS[model_display_name]

#TensorFlow Hub 에서 선택한 모델 로드

In [8]:
print('loading model...')
hub_model = hub.load(model_handle)
print('model loaded!')

loading model...
model loaded!


# CV2 에러 방지를 위한 최신 업데이트

In [10]:
pip install opencv-python-headless==4.5.2.52

     |████████████████████████████████| 38.2 MB 1.3 MB/s 
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.5.5.64
    Uninstalling opencv-python-headless-4.5.5.64:
      Successfully uninstalled opencv-python-headless-4.5.5.64


#영상 넣고 결과 영상 만들기

In [12]:
import cv2

video_input_path = "/content/팔척귀신녀.mp4"
# linux에서 video output의 확장자는 반드시 avi 로 설정 필요. 
video_output_path = "/content/영상고치기.avi"

# 동영상 불러오기
cap = cv2.VideoCapture(video_input_path)

# avi 코덱 선언
codec = cv2.VideoWriter_fourcc(*'XVID')
# 동영상 이미지 가로 크기
width = round(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
# 동영상 이미지 세로 크기
height = round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
vid_size = (round(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
# 동영상 초당 영상 재생 개수
vid_fps = cap.get(cv2.CAP_PROP_FPS)
# 동영상 저장용 선언
vid_writer = cv2.VideoWriter(video_output_path, codec, vid_fps, vid_size) 

# 동영상 총 이미지 갯수=총 frame 갯수
frame_cnt = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print('총 Frame 갯수:', frame_cnt)
print('fps:',vid_fps)
print('총 시간:', int(frame_cnt/vid_fps),'초 = ', int(frame_cnt/vid_fps/60),'분', int(frame_cnt/vid_fps)%60,'초')
print('가로:', width, '세로:', height)

# 동영상에서 불필요한 장면을 스킵 하기 위해서 비디오 시작지점을 설정한다.
# cap.set(cv2.CAP_PROP_POS_FRAMES, 40*vid_fps)

총 Frame 갯수: 271
fps: 30.0
총 시간: 9 초 =  0 분 9 초
가로: 406 세로: 720


In [13]:
import time

In [14]:
def get_hub_model_result(image, _model):
  # _model은 위에서 선택한 모델로 불러오도록 코드를 구성
  # 선택한 모델로 찾을 image와 box와 labeling할 이미지  
  start = time.time()
  results = _model(image)
  end = time.time()
  print(end-start)

  # different object detection models have additional results
  # all of them are explained in the documentation
  # tensorflow hub의 사용법은 모델의 결과값을 dict형태로 선언해서 사용하기 때문에 아래 형태로 선언
  result = {key:value.numpy() for key,value in results.items()}
  # 시작이 1 인지 0 인지 차이를 선언
  label_id_offset = 0

  viz_utils.visualize_boxes_and_labels_on_image_array(
      image[0],  # 함수 내부에서 원본 이미지의 크기 확인 및 화면에 그리는 이미지 선언
                 # 모델 종류에 상관없이 이미지 크기는 원본 그대로 넣어도 된다.
                 # 자동으로 /255 계산과, resize 코드가 내장되어 있다.
                 # 원본 이미지값을 사용해야 나중에 화면에 표시할때 원본 크기로 계산할때 필요하기 때문
      result['detection_boxes'][0],  # 사각형 좌표 들
      (result['detection_classes'][0] + label_id_offset).astype(int),  # class 종류
      result['detection_scores'][0],  # 각 사각형에 대한 object인 확률 (0~1 사이)
      category_index,  # category 종류를 넣어주면 출력한 class 숫자를 category_index[class]로 출력해준다.
      use_normalized_coordinates=True, # max_boxes_to_draw와 min_score_thresh 사용여부
      max_boxes_to_draw=200,  # 최대 200개 까지 그린다.
      min_score_thresh=.30,  # score값이 0.3 이하는 제거
      agnostic_mode=False,  # True일 경우 score만 표시, clas 표시 안함, 기본은 False
      # 아래는 해당 값이 있으면 사용함
   ) 
  return image[0]

In [15]:
frame_count = 0
while True:
  hasFrame, img_frame = cap.read()
  print('frame count', frame_count)  
  if not hasFrame:
    print('더 이상 처리할 frame이 없습니다.')
    break  

  # hub_model에서는 이미지는 4체널로 선언해줘야 하기 때문에 앞에 1차원을 만들어줌
  test_img = np.array(img_frame).reshape((1, height, width, 3)).astype(np.uint8)
  # test_img : 테스트할 이미지, hub_model : 테스트할 모델
  draw_img_frame  = get_hub_model_result(test_img, hub_model)
  # 동영상중에서 1개의 frame 이미지를 진행했다는 count 표시용
  frame_count += 1

  # 아래는 확인용 코드
  # plt.figure(figsize=(24,32))  # 크게 보고싶으면 코드 주석 제거
  # plt.imshow(draw_img_frame)
  # plt.show()
  vid_writer.write(draw_img_frame)  

  # 너무 많이 저장하면 오래걸리기 때문에 짧은 시간안에 테스트 할정도로 설정
  if frame_count > 100: break  

vid_writer.release()
cap.release()

frame count 0
12.293290853500366
frame count 1
4.236741542816162
frame count 2
4.327742576599121
frame count 3
4.178061485290527
frame count 4
4.260885238647461
frame count 5
4.297652244567871
frame count 6
4.251604318618774
frame count 7
4.3075830936431885
frame count 8
4.309257745742798
frame count 9
4.232635974884033
frame count 10
4.234773635864258
frame count 11
4.479790687561035
frame count 12
4.193079471588135
frame count 13
4.199892997741699
frame count 14
4.151808023452759
frame count 15
4.316534996032715
frame count 16
4.265373706817627
frame count 17
4.192974328994751
frame count 18
4.248050928115845
frame count 19
4.206481456756592
frame count 20
4.2756028175354
frame count 21
4.229045867919922
frame count 22
4.223596811294556
frame count 23
4.177287578582764
frame count 24
4.2590651512146
frame count 25
4.2052836418151855
frame count 26
4.206609010696411
frame count 27
4.3054327964782715
frame count 28
4.23715615272522
frame count 29
4.186382532119751
frame count 30
4.2361